## Assignment

The task is to build and train a classifier given a labeled dataset and then use it to infer the labels of a given unlabeled evaluation dataset. 

You will find the training and evaluation data on canvas.

Here's the training data: TrainOnMe-2.csv 

Here's the evaluation data: EvaluateOnMe-2.csv 

Here's the ground truth: EvaluationGT-2.csv

You can use whatever python libraries you like! The steps below are suggestions, but feel free to try any other techniques we discussed in class.

You can submit the predicted labels by uploading them in csv format, which will then be compared to the ground truth.


In [1]:
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Import packages 
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# For feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# For min-max scaling
from sklearn.preprocessing import MinMaxScaler

# For encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Some models you can try
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier


## Load the training and evaluation datasets

In [16]:
# Read datasets
df = pd.read_csv('TrainOnMe-2.csv')
eval_df = pd.read_csv('EvaluateOnMe-2.csv')

## Data pre-processing

In [17]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# 1. Impute NA values

numerical_imputer = SimpleImputer(strategy = "median")
numerical_data = df.select_dtypes(include = ["float64"])
numerical_data_imputed = numerical_imputer.fit_transform(numerical_data)
numerical_data_imputed_df = pd.DataFrame(numerical_data_imputed)

categorical_imputer = SimpleImputer(strategy = "most_frequent")
categorical_data = df.select_dtypes(include = ["object"])
categorical_data_imputed = categorical_imputer.fit_transform(categorical_data)
categorical_data_imputed_df = pd.DataFrame(categorical_data_imputed)

na_replaced_df = pd.concat([categorical_data_imputed_df, numerical_data_imputed_df], axis = 1)
na_replaced_df.columns = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14"]

# 2. Split the training data into features and labels

features = na_replaced_df.drop("1", axis = 1)
numerical_features_df = pd.DataFrame(features.select_dtypes(include = ["float64"]))
categorical_features_df = pd.DataFrame(features.select_dtypes(include = ["object"]))
labels_prepared = na_replaced_df["1"].copy()

# 3. Encode categorical data

one_hot_encoder = OneHotEncoder()
categorical_features_encoded = one_hot_encoder.fit_transform(categorical_features_df)
categorical_features_encoded_df = pd.DataFrame(categorical_features_encoded.toarray())

## Dealing with outliers

In [21]:
# Try to remove numerical outliers from training data to improve performance
# There are different ways to do this but one way could be to use stats.zscore

z_scores = stats.zscore(numerical_features_df)
outlier_threshold = 3.0
numerical_outliers = np.abs(z_scores) > outlier_threshold
filtered_numerical_features = numerical_features_df[~ numerical_outliers]
outliers_removed_numerical_features_imputed = numerical_imputer.fit_transform(filtered_numerical_features)
outliers_removed_numerical_features_imputed_df = pd.DataFrame(outliers_removed_numerical_features_imputed)

## Scaling the features

In [23]:
# Scale your features
# You can try both standardscaler and minmaxscaler and see which works better

#from sklearn.preprocessing import StandardScaler
#standard_scaler = StandardScaler()

min_max_scaler = MinMaxScaler()
numerical_features_scaled = min_max_scaler.fit_transform(outliers_removed_numerical_features_imputed_df)
numerical_features_scaled_df = pd.DataFrame(numerical_features_scaled)

# Prepare finalized features
features_prepared = pd.concat([categorical_features_encoded_df, numerical_features_scaled_df], axis = 1)

## Feature selection

In [ ]:
# You could try to apply SelectKBest class to extract the most useful features (this is optional but MIGHT improve accuracy)
# Remove whichever features that are not useful



## Split your data to train and test set

In [28]:
# Train, test, split
X_train, X_test, y_train, y_test = train_test_split(features_prepared, labels_prepared, 
                                                    train_size = 0.9, random_state = 0)

## Fit the model

* You can try models other than the models listed below
* You can try different hyperparameters
* Evaluate your model using cross-validation

In [25]:
# Try linear SVM classifier
linear = SVC(kernel = "linear", C = 0.5).fit(X_train, y_train)
print(linear.score(X_test, y_test))

# Evaluate using cross-validation
scores = cross_val_score(linear, X_test,y_test, cv = 5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.5445544554455446
0.41 accuracy with a standard deviation of 0.12


In [26]:
#Try decision tree classifier
decision_tree = DecisionTreeClassifier(criterion = "gini").fit(X_train, y_train)
print(decision_tree.score(X_test,y_test))

# Evaluate using cross-validation
scores = cross_val_score(decision_tree, X_test, y_test, cv = 10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.504950495049505
0.60 accuracy with a standard deviation of 0.15


In [27]:
#Try random forest classifier
random_forest = RandomForestClassifier().fit(X_train, y_train)
print(random_forest.score(X_test,y_test))

scores = cross_val_score(random_forest,X_test,y_test,cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.5742574257425742
0.51 accuracy with a standard deviation of 0.16


In [65]:
# Use your best model to predict the labels for the evaluation set

#best_model = DecisionTreeClassifier(criterion = "gini").fit(X_eval)
#y_pred = best_model.predict(X_eval)
#y_pred = decision_tree.predict()

#print(y_pred)



In [599]:
# Save your predictions to a CSV and upload it to Canvas

pd.DataFrame(y_pred).to_csv("file.txt", index = False, header = False)